In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)
    
    def edge_attention(self, edges):
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        return {'e': F.leaky_relu(a)}
    
    def message_func(self, edges):
        return {'z': edges.src['z'], 'e':edges.data['e']}
    
    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}
        
    def forward(self, h):
        z = self.fc(h)
        self.g.ndata['z'] = z
        self.g.apply_edges(self.edge_attention)
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')

In [22]:
class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, num_heads, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
        self.merge = merge

    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            # concat on the output feature dimension (dim=1)
            return torch.cat(head_outs, dim=1)
        else:
            # merge using average
            return torch.mean(torch.stack(head_outs))

In [23]:
class GAT(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1 = MultiHeadGATLayer(g, in_dim, hidden_dim, num_heads)
        # Be aware that the input dimension is hidden_dim*num_heads since
        # multiple head outputs are concatenated together. Also, only
        # one attention head in the output layer.
        self.layer2 = MultiHeadGATLayer(g, hidden_dim * num_heads, out_dim, 1)

    def forward(self, h):
        h = self.layer1(h)
        h = F.elu(h)
        h = self.layer2(h)
        return h

In [18]:
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh
import networkx as nx

def load_cora_data():
    data = citegrh.load_cora()
    features = torch.FloatTensor(data.features)
    labels = torch.LongTensor(data.labels)
    mask = torch.BoolTensor(data.train_mask)
    g = data.graph
    # add self loop
    g.remove_edges_from(nx.selfloop_edges(g))
    g = DGLGraph(g)
    g.add_edges(g.nodes(), g.nodes())
    return g, features, labels, mask

In [19]:
import time
import numpy as np

g, features, labels, mask = load_cora_data()

In [24]:
# create the model, 2 heads, each head has hidden size 8
net = GAT(g,
          in_dim=features.size()[1],
          hidden_dim=8,
          out_dim=7,
          num_heads=2)

# create optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

# main loop
dur = []
for epoch in range(30):
    if epoch >= 3:
        t0 = time.time()

    logits = net(features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[mask], labels[mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
        epoch, loss.item(), np.mean(dur)))

/Users/Tree/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Tree/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 1.9459 | Time(s) nan
Epoch 00001 | Loss 1.9452 | Time(s) nan
Epoch 00002 | Loss 1.9444 | Time(s) nan
Epoch 00003 | Loss 1.9436 | Time(s) 0.0932
Epoch 00004 | Loss 1.9428 | Time(s) 0.1007
Epoch 00005 | Loss 1.9420 | Time(s) 0.0962
Epoch 00006 | Loss 1.9412 | Time(s) 0.0941
Epoch 00007 | Loss 1.9404 | Time(s) 0.0929
Epoch 00008 | Loss 1.9395 | Time(s) 0.0912
Epoch 00009 | Loss 1.9387 | Time(s) 0.0903
Epoch 00010 | Loss 1.9378 | Time(s) 0.0887
Epoch 00011 | Loss 1.9370 | Time(s) 0.0881
Epoch 00012 | Loss 1.9361 | Time(s) 0.0878
Epoch 00013 | Loss 1.9352 | Time(s) 0.0875
Epoch 00014 | Loss 1.9343 | Time(s) 0.0869
Epoch 00015 | Loss 1.9333 | Time(s) 0.0867
Epoch 00016 | Loss 1.9324 | Time(s) 0.0860
Epoch 00017 | Loss 1.9314 | Time(s) 0.0856
Epoch 00018 | Loss 1.9304 | Time(s) 0.0854
Epoch 00019 | Loss 1.9295 | Time(s) 0.0852
Epoch 00020 | Loss 1.9284 | Time(s) 0.0851
Epoch 00021 | Loss 1.9274 | Time(s) 0.0851
Epoch 00022 | Loss 1.9264 | Time(s) 0.0848
Epoch 00023 | Loss 1